<a href="https://colab.research.google.com/github/toandaominh1997/Kaggle/blob/master/Mercedes_Benz_Greener_Manufacturing/Mercedes_Benz_Greener_Manufacturing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import pandas as pd
import numpy as np
URL = "drive/My Drive/Dataset/Mercedes-Benz Greener Manufacturing/"

In [0]:
train = pd.read_csv(os.path.join(URL, "train.csv"))
test = pd.read_csv(os.path.join(URL, "test.csv"))


In [73]:
train_y = train['y'].values
test_id = test['ID'].values
test_X = test.drop(columns=['ID'])

train_X = train.drop(columns=['ID', 'y'])
print(train_X.shape)
print(train_y.shape)

(4209, 376)
(4209,)


In [74]:
from sklearn import preprocessing

pre = preprocessing.LabelEncoder()
for i in range(0, 9):
  train_X.iloc[:, i] = pre.fit_transform(train_X.iloc[:, i])
  test_X.iloc[:, i] = pre.fit_transform(test_X.iloc[:, i])
train_X.head()

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,32,23,17,0,3,24,9,14,0,0,...,0,0,1,0,0,0,0,0,0,0
1,32,21,19,4,3,28,11,14,0,0,...,1,0,0,0,0,0,0,0,0,0
2,20,24,34,2,3,27,9,23,0,0,...,0,0,0,0,0,0,1,0,0,0
3,20,21,34,5,3,27,11,4,0,0,...,0,0,0,0,0,0,0,0,0,0
4,20,23,34,5,3,12,3,13,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
print(train_X.shape)
print(test_X.shape)

(4209, 376)
(4209, 376)


In [76]:
!pip install xgboost

In [77]:
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=0.2, random_state=4242)

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)

params = {}
params['objective'] = 'reg:linear'
params['eta'] = 0.02
params['max_depth'] = 4

def xgb_r2_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels, preds)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

clf = xgb.train(params, d_train, 1000, watchlist, early_stopping_rounds=50, feval=xgb_r2_score, maximize=True, verbose_eval=10)

[0]	train-rmse:99.1397	valid-rmse:98.2538	train-r2:-58.3426	valid-r2:-67.6247
Multiple eval metrics have been passed: 'valid-r2' will be used for early stopping.

Will train until valid-r2 hasn't improved in 50 rounds.
[10]	train-rmse:81.1832	valid-rmse:80.2714	train-r2:-38.7928	valid-r2:-44.804
[20]	train-rmse:66.541	valid-rmse:65.5967	train-r2:-25.7332	valid-r2:-29.5876
[30]	train-rmse:54.6149	valid-rmse:53.6305	train-r2:-17.0092	valid-r2:-19.4459
[40]	train-rmse:44.9172	valid-rmse:43.8842	train-r2:-11.1814	valid-r2:-12.6899
[50]	train-rmse:37.0508	valid-rmse:35.9587	train-r2:-7.28831	valid-r2:-8.19158
[60]	train-rmse:30.6912	valid-rmse:29.5297	train-r2:-4.68722	valid-r2:-5.19867
[70]	train-rmse:25.5738	valid-rmse:24.3406	train-r2:-2.94878	valid-r2:-3.21158
[80]	train-rmse:21.4816	valid-rmse:20.1727	train-r2:-1.78616	valid-r2:-1.89273
[90]	train-rmse:18.2374	valid-rmse:16.8512	train-r2:-1.00815	valid-r2:-1.01856
[100]	train-rmse:15.6927	valid-rmse:14.2334	train-r2:-0.486848	valid-r2:

In [78]:
from sklearn.metrics import r2_score

print("Training score: ", r2_score(clf.predict(d_train), y_train))
print("Valid score: ", r2_score(clf.predict(d_valid), y_valid))

Training score:  0.2284065476204835
Valid score:  0.49802579263943847


In [0]:
res = pd.DataFrame()
res['ID']= test_id
res['y'] = clf.predict(xgb.DMatrix(test_X))

In [0]:
res.to_csv('xgb.csv', index=False)